# Scraping images from mapbox satelitar view  
Mapbox Free tier allows 50,000 requests per month : consider logging how many images have been scraped already to not exceed this limit.  


Pipeline : 
- Define area of interest (bounding box) from coordinates (lat, lon)
- Get the satelitar view of the bounding box from mapbox API
- Crop the images to remove watermark
- Save the images to disk
- Get the bounding boxes of buildings from OSM for the area of interest
- Save the bounding boxes to disk
- Clean overlapping bounding boxes (remove smaller ones or merge them)
- Save the cleaned bounding boxes to disk

The areas of interest have been picked manually :
- Top 20 most populous cities in France
- 100 rural areas coordinates picked randomly across France
- Maybe some random forest to get images without buildings (negative samples)

For the cities : 
- We define a bounding box around the city center with a fixed size (e.g., 10km x 10km)
- We then take random points from this bounding box to scrape images

Then, we can use the pipeline with these points to scrape images and get bounding boxes of buildings.

importing packages

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

CONFIG

In [2]:
# Load environment variables from .env file
load_dotenv()

# MAPBOX
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

# Image parameters
WIDTH = 512
HEIGHT = 512 + 30  # extra space to then delete bottom part with watermark
ZOOM = 18  # 18-19 to see buildings, swimming pools clearly

PATH = Path().cwd().parent
BASE_DIR = PATH / "data"
RAW_DIR = BASE_DIR / "raw_images"
CROPPED_DIR = BASE_DIR / "cropped_images"

RAW_POLYGON_DIR = BASE_DIR / "raw_polygons"
CLEANED_POLYGON_DIR = BASE_DIR / "cleaned_polygons"

sys.path.append(str(PATH))

First, we get all the coordinates of the areas of interest from the two csv files.

In [3]:
# to import csv files
import pandas as pd

Rural areas coordinates file : "rural_coords.csv"

In [4]:
FILE_NAME: str = "rural_coords.csv"
rural_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ",",
    header=None
)
rural_coords.columns = ["latitude", "longitude"]

In [5]:
# checking the rural coordinates file
print(f"Number of rural areas: {len(rural_coords)}")
rural_coords.head()

Number of rural areas: 100


,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


Cities coordinates file : "city_coords.csv"

In [6]:
FILE_NAME: str = "city_coords.csv"
city_coords: pd.DataFrame = pd.read_csv(
    PATH / FILE_NAME,
    sep = ","
)

In [7]:
# checking the city coordinates file
print(f"Number of cities: {len(city_coords)}")
city_coords.head()

Number of cities: 19


,city,lat,lng
0,Paris,48.8567,2.3522
1,Bordeaux,44.8400,-0.5800
2,Marseille,43.2964,5.3700
3,Lyon,45.7600,4.8400
4,Toulouse,43.6045,1.4440


For the cities, we now create a bounding box around the city center with a fixed size (e.g., 10km x 10km) and then take random points from this bounding box to scrape images.

In [8]:
from miscellaneous import create_bbox_from_city_coordinates

city_coords_bbox: dict[str, dict[str, float]] = {}
# {city :  {"west": .., "south": .., "east": .., "north": ..}

for _, row in city_coords.iterrows():
    city_name: str = row["city"]
    latitude: float = row["lat"]
    longitude: float = row["lng"]
    bbox: dict[str, float] = create_bbox_from_city_coordinates(
        latitude,
        longitude,
        width=10,
        height=10
    )
    # width and height are in km
    city_coords_bbox[city_name] = bbox


In [9]:
city_coords_df = pd.DataFrame.from_dict(city_coords_bbox, orient="index")
city_coords_df.head()

,west,south,east,north
Paris,2.284676,48.811347,2.419842,48.902013
Bordeaux,-0.640288,44.792973,-0.519611,44.886994
Marseille,5.306649,43.252774,5.433447,43.339990
Lyon,4.774246,45.716018,4.905859,45.803944
Toulouse,1.383371,43.558618,1.504726,43.650350


Now that we have the bounding boxes for each city, we can get random points from these bounding boxes to scrape images.

In [23]:
from miscellaneous import get_random_points_in_bbox

city_random_points: list[dict[str, float]] = []
for city, bbox in city_coords_bbox.items():
    random_points: list[dict[str, float]] = get_random_points_in_bbox(
        bbox,
        number_of_points=10
    )
    # 10 coordinates/images for each city -> 20 * 10 = 200 images
    city_random_points.extend(random_points)
city_random_points_df = pd.DataFrame(city_random_points)
city_random_points_df.columns = ["latitude", "longitude"]

Now that we have all the coordinates (rural + cities), we combine them into a single list of coordinates to use in the pipeline.

In [30]:
# we combine both rural and city coordinates into a single dataframe
all_coords_df = pd.DataFrame(columns=["latitude", "longitude"])
all_coords_df = pd.concat(
    [
        rural_coords,
        city_random_points_df
    ],
    ignore_index=True
)

In [31]:
all_coords_df.head()

,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


We can now use the pipeline to scrape images and get bounding boxes of buildings.

In [ ]:
from miscellaneous import create_bbox_from_coordinates

for _, row in all_coords_df.sample(5).iterrows():
    print(f"Latitude: {row['latitude']}, Longitude: {row['longitude']}")
    # now we get bbox from coordinates
    bbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude']
    )
    # other parameters include :
    # - img_height: int = 512 + 30,
    # - img_width: int = 512,
    # - pixel_size: float = 0.4
    # returns : {"west": .., "south": .., "east": .., "north": ..}
    print(f"BBox: {bbox}\n")


ModuleNotFoundError: No module named 'utils'